# Import libraries

In [ ]:
import os

import gcsfs
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

In [ ]:
gfs = gcsfs.GCSFileSystem(project="smle-attribution-d237")

# Define variables

In [ ]:
root = "gs://hm-images-bucket/"
img_path = "images"
annotation_path = "annotations"

# Image counts

In [ ]:
total = 0
for base, dirs, files in gfs.walk(os.path.join(root, img_path)):
    count = len(files)
    total += count
    print(f"{base}: {count}")
print(f"Total: {total}")

# Create data

In [ ]:
num = len(os.path.join(root, img_path)) - 4

file_list = gfs.glob(root + "images/**/*.jpg", recursive=True)
castor_list = [int(os.path.basename(path)[:-4]) for path in file_list]
path_list = [path[num:] for path in file_list]
print(len(file_list))

In [ ]:
df = pd.DataFrame(data=
    {
        "path": path_list,
        "castor": castor_list
    }
)
print(df.shape)
df.head()

In [ ]:
# df.to_csv(os.path.join(root, annotation_path, "castors.csv"), index=False)

# Padma table

In [ ]:
padma = pd.read_parquet("gs://hdl-tables/dma/product_article_datamart",
                        columns=["product_code", "article_code", "castor"])
print(padma.shape)
padma.head()

In [ ]:
padma.isna().any()

In [ ]:
padma = padma.drop_duplicates()
padma.shape

In [ ]:
padma.isna().any()

In [ ]:
padma.dtypes

In [ ]:
padma.castor = padma.castor.astype(int)
padma.dtypes

# PIM table

In [ ]:
pim = pd.read_parquet("gs://hdl-tables/dim/dim_pim",
                      columns=["product_code", "article_code", "product_fit"])
print(pim.shape)
pim.head()

In [ ]:
pim.isna().any()

In [ ]:
pim.isna().sum()

In [ ]:
pim = pim.dropna(axis=0, subset=["article_code", "product_fit"])
print(pim.shape)
pim.head()

In [ ]:
pim["product_fit"].value_counts()

In [ ]:
pim = pim.drop_duplicates()
pim.shape

In [ ]:
pim.isna().any()

# Merge pim, padma

In [ ]:
data = pim.merge(padma, on=["product_code", "article_code"], how="left")
print(data.shape)
data.head()

In [ ]:
data.dtypes

In [ ]:
data = data[~data["product_fit"].str.contains("[", regex=False)]
print(data.shape)
data.head()

In [ ]:
data.isna().any()

In [ ]:
data = data.drop(axis=1, labels=["product_code", "article_code"])
data.shape

# Merge with castors

In [ ]:
df.dtypes

In [ ]:
out = df.merge(data, on="castor", how="inner")
print(out.shape)
out.head()

In [ ]:
out.isna().any()

In [ ]:
out.dtypes

In [ ]:
out.castor.unique().shape

In [ ]:
out["product_fit"].value_counts()

In [ ]:
out["labels"] = out["product_fit"].astype("category").cat.codes
print(out.shape)
out.head()

In [ ]:
out.labels.value_counts()

# Split data

In [ ]:
cv = StratifiedGroupKFold(n_splits=2)

In [ ]:
train_idxs, test_idxs = next(cv.split(out.path, out.labels, out.castor))
print("TRAIN:", out.castor[train_idxs], out.labels[train_idxs])
print(" TEST:", out.castor[test_idxs], out.labels[test_idxs])

In [ ]:
len(train_idxs)

In [ ]:
len(test_idxs)

# Split data 2

In [ ]:
tmp = out[["product_fit", "castor"]].drop_duplicates()
print(tmp.shape)
tmp.head()

In [ ]:
sub_train = tmp.groupby("product_fit").sample(frac=0.8)
sub_train["is_train"] = True
print(sub_train.shape)
sub_train.head()

In [ ]:
final = out.merge(sub_train[["castor", "is_train"]], on="castor", how="left")
print(final.shape)
final.head()

In [ ]:
final.isna().any()

In [ ]:
final.fillna(False, inplace=True)

In [ ]:
final.isna().any()

In [ ]:
final.dtypes

In [ ]:
final.is_train.sum()

In [ ]:
(~final.is_train).sum()

In [ ]:
train = final.loc[final.is_train, ["path", "castor", "product_fit", "labels"]]
train.shape

In [ ]:
test = final.loc[~final.is_train, ["path", "castor", "product_fit", "labels"]]
test.shape

In [ ]:
train.product_fit.value_counts()

In [ ]:
test.product_fit.value_counts()

In [ ]:
assert not set(train.castor) & set(test.castor)

# Write data

In [ ]:
train_fit = out.iloc[train_idxs, :]
test_fit = out.iloc[test_idxs, :]

In [ ]:
print(train_fit.shape)
print(test_fit.shape)

In [ ]:
# out.to_csv(os.path.join(root, annotation_path, "full_fit.csv"), index=False)

# train_fit.to_csv(os.path.join(root, annotation_path, "train.csv"), index=False)
# test_fit.to_csv(os.path.join(root, annotation_path, "test.csv"), index=False)

In [ ]:
out_gcp = out[["path", "product_fit"]].copy()
out_gcp["path"] = "gs://hm-images-bucket/images/" + out_gcp["path"]

In [ ]:
out_gcp["mode"] = "VALIDATION"
out_gcp.loc[train_idxs, "mode"] = "TRAINING"

In [ ]:
out_gcp = out_gcp[["mode", "path", "product_fit"]]
out_gcp

In [ ]:
out_gcp.shape

In [ ]:
# out_gcp.to_csv(os.path.join(root, annotation_path, "full_fit_gcai.csv"), index=False, header=False)